In [1]:
import pandas as pd
import pandas as pd
import numpy as np
from tqdm import notebook
import ast
import re
from fuzzywuzzy import fuzz
from datetime import datetime, timedelta
import spacy

In [2]:
path = 'Completed Batches/'
subset1_pat = pd.read_json(path + 'batch1_patrick_result.json')
subset2_pat = pd.read_json(path + 'batch2_patrick_result.json')
subset3_pat = pd.read_json(path + 'batch3_patrick_result.json')
subset4_pat = pd.read_json(path + 'batch4_patrick_result.json')
subset5_pat = pd.read_json(path + 'batch5_patrick_result.json')
subset1_thom = pd.read_json(path + 'batch1_thom_result.json')
subset1_chiel = pd.read_json(path + 'batch1_chiel_result.json')
subset2_chiel = pd.read_json(path + 'batch2_chiel_result.json')
subset3_chiel = pd.read_json(path + 'batch3_chiel_result.json')
subset1_bar = pd.read_json(path + 'batch1_result.json')
subset2_bar = pd.read_json(path + 'result.json')
subset3_bar = pd.read_json(path + 'batch2_result.json')
subset4_bar = pd.read_json(path + 'batch3_result.json')
subset5_bar = pd.read_json(path + 'batch4_result.json')
subset6_bar = pd.read_json(path + 'batch2_thom_result.json')

#### Correcties:
Chiel:
- Christiaan Andriesz 1416
- Jan de Bruijn 1463
- Abraham van de Heuvel (1823 super interessant twijfel geval)
- Pieter van Essen (1825 super interessant twijfel geval)
- Hans Pietersz 3792

Patrick:
- Jan Smit 4194 (zelde als de interessante gevallen van Chiel)
- Hendrik van Alen 593 (zelfde schip maar naam is net even anders)

In [3]:
subset1_pat.at[593, 'vocop_match'] = 0
subset1_chiel.at[1416,'vocop_match'] = 0
subset1_chiel.at[1463,'vocop_match'] = 0
subset2_chiel.at[3729, 'vocop_match'] = 0
subset4_bar.at[9025, 'vocop_match'] = ('Johannis Oosterhoff', 140442)
subset5_bar.at[9927, 'vocop_match'] = ('Wiggert Andresz', 374198)
subset6_bar.at[2670, 'vocop_match'] = ('Johan Christiaan Richter', 379155)
subset6_bar.at[2688, 'vocop_match'] = ('Jacob Meijer', 178549)
subset6_bar.at[2689, 'vocop_match'] = ('Juriaan Bartels', 178548)
subset6_bar.at[2690, 'vocop_match'] = ('Willem Borsenius', 283179)
subset6_bar.at[2691, 'vocop_match'] = ('Jan de Vaij', 182941)

In [4]:
final_pat = subset1_pat
final_pat = final_pat.append(subset2_pat)
final_pat = final_pat.append(subset3_pat).append(subset4_pat).append(subset5_pat)

final_chiel = subset1_chiel
final_chiel = final_chiel.append(subset2_chiel)
final_chiel = final_chiel.append(subset3_chiel)

final_barry = subset1_bar
final_barry = final_barry.append(subset2_bar).append(subset3_bar).append(subset4_bar).append(subset5_bar).append(subset6_bar)

final_df = final_pat.append(final_chiel).append(subset1_thom).append(final_barry)

In [6]:
final_df = final_df.loc[~final_df.index.duplicated(keep='first')]
final_df[final_df.vocop_match != 0]

,uuid,rubriek,notaris,inventarisNr,akteNr,akteType,datering,taal,beschrijving,namen,urls,text,name,data_matches,data_entry,vocop_match
372,2dd7b308-e622-01b5-22ae-abdeba00fed2,358,JAN VERLEIJ,11889,11294,Machtiging,1744-07-02,nederlands,\nschip Nieuland; schip Zaamslag; schip Waterv...,"[{'voornaam': 'Jurriaan', 'tussenvoegsel': Non...","['KLAB05459000335.JPG', 'KLAB05459000336.JPG',...",No: 259 Grseter gepasseert den 2e: Julij 1744J...,Christiaan Morgenfeld,"[{'index': 44997, 'name_original': 'Christiaan...","{'44997': {'ships': [], 'rank': [], 'location'...","[Christiaan Morgenfeld, 298101]"
375,2dd7b308-e622-01b5-22ae-abdeba00fed2,358,JAN VERLEIJ,11889,11294,Machtiging,1744-07-02,nederlands,\nschip Nieuland; schip Zaamslag; schip Waterv...,"[{'voornaam': 'Jurriaan', 'tussenvoegsel': Non...","['KLAB05459000335.JPG', 'KLAB05459000336.JPG',...",No: 259 Grseter gepasseert den 2e: Julij 1744J...,Nanning Broeder,"[{'index': 68473, 'name_original': 'Nanning Br...","{'68473': {'ships': ['Zaamslag', 'ZAAMSLAG'], ...","[Nanning Broeder, 68473]"
376,2dd7b308-e622-01b5-22ae-abdeba00fed2,358,JAN VERLEIJ,11889,11294,Machtiging,1744-07-02,nederlands,\nschip Nieuland; schip Zaamslag; schip Waterv...,"[{'voornaam': 'Jurriaan', 'tussenvoegsel': Non...","['KLAB05459000335.JPG', 'KLAB05459000336.JPG',...",No: 259 Grseter gepasseert den 2e: Julij 1744J...,Jan Fredrik Stephaan,"[{'index': 256327, 'name_original': 'Jan Fredr...","{'256327': {'ships': [], 'rank': [], 'location...","[Jan Fredrik Stephaen, 289548]"
509,ee21c8bc-bfea-25ee-c6c1-ddf74644012b,358,JAN VERLEIJ,11890,12427,Obligatie,1744-09-12,nederlands,\nschip Diemen\n,"[{'voornaam': 'Pieter', 'tussenvoegsel': None,...","['KLAB05439000262.JPG', 'KLAB05439000263.JPG',...",Ragd werlke Getijnke Gepasseert Den 12 feptemb...,Pieter Klijn,"[{'index': 2148, 'name_original': 'Pieter Clin...","{'2148': {'ships': [], 'rank': [], 'location':...","[Pieter Klijn, 284049]"
563,35b32066-d1ee-0b29-2180-38584da36f12,358,JAN VERLEIJ,11891,11348,Obligatie,1744-10-29,nederlands,"\nschip d' Anna, VOC, 300 gulden\n","[{'voornaam': 'Nomme', 'tussenvoegsel': None, ...","['KLAB05426000116.JPG', 'KLAB05426000117.JPG',...",No: 636 Puijete Guijfiet Den 29 Eebber 1744Nte...,Cornelis Christiaansz,"[{'index': 13740, 'name_original': 'Cornelis C...","{'13740': {'ships': [], 'rank': [], 'location'...","[Cornelis Christiaansz, 329326]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2749,b8eafa41-8233-d58f-1022-092f1a8a69ee,358,JAN VERLEIJ,11915,15216,Testament,1751-11-15,nederlands,\nSchip de Immegonda\n,"[{'voornaam': 'Grietje', 'tussenvoegsel': None...","['KLAB05657000441.JPG', 'KLAB05657000442.JPG',...",Calkoen L Dit Testament van man & vrouw vervat...,Michiel Hendrik Kerken,"[{'index': 297296, 'name_original': 'Michiel H...","{'297296': {'ships': [], 'rank': [], 'location...","[Michiel Hendrik Kerke, 409527]"
2834,1420a408-ff1f-a7c4-6092-db5d4e4e5212,358,JAN VERLEIJ,11915,18096,Machtiging,1751-10-09,nederlands,"\nschipper schip de Jager, VOC kamer Hoorn\n","[{'voornaam': 'Govert', 'tussenvoegsel': None,...","['KLAB05657000178.JPG', 'KLAB05657000179.JPG',...",No: 371 Procuratie gecasseerd den 4e: October ...,Jacob Leertouwen,"[{'index': 268969, 'name_original': 'Jacob Lee...","{'268969': {'ships': [], 'rank': [], 'location...","[Jacob Leertouwer, 311020]"
2880,0f0f6da3-5628-3b6a-c33d-6104c6ade355,358,JAN VERLEIJ,11915,20001,Machtiging,1751-12-28,nederlands,\nschip Crabbendijk\n,"[{'voornaam': 'Johan', 'tussenvoegsel': 'van',...","['KLAB05657000615.JPG', 'KLAB05657000616.JPG',...",No: 505 Procuratie gecasseerd den 28e: decembr...,Johan van Veldhuijsen,"[{'index': 418944, 'name_original': 'Johan van...","{'418944': {'ships': ['Krabbendijke'], 'rank':...","[Johan van Velthuijsen, 418944]"
3018,4fbc09ce-5afe-34e3-2ab4-d323cb49722d,358,JAN VERLEIJ,11918,13012,Obligatie,1752-10-13,nederlands,"\nstuurman schip ""Bloemendaal"" OIC\n","[{'voornaam': 'Hendrik Coenraad', 'tussenvoegs...","['KLAB05633000172.J

In [ ]:
nlp = spacy.load('nl_core_news_sm', disable=['parser', 'tagger', 'textcat'])

In [76]:
def multimatch(start, end, true, prev, distance):
    if true == []:
        return (start, end)
    if prev.i == len(prev.doc) - 1:
        return (start, end)
    if fuzz.ratio(true[0], prev.nbor().text) >= distance:
        return multimatch(start, prev.nbor().idx + len(prev.nbor()), true[1:], prev.nbor(), distance)
    else:
        return (start, end)

def singlematch(start, end, true, prev, distance):
    newtoken = prev.text + ' ' + prev.nbor().text
    if fuzz.ratio(true, newtoken) >= distance:
        return (start, start + len(newtoken) + 1)
    else:
        return (start, end)

def entity_finder(row, distance, true):
    doc = nlp(row.text)
    holder = []
    for token in doc:
        for x in true:
            if x != None:
                split = x.split(' ')

                if fuzz.ratio(x, token.text) >= distance:
                    holder.append(x)
                    
                elif fuzz.ratio(split[0], token.text) >= distance:
                    if len(split) > 1:
                        result = multimatch(token.idx, token.idx + len(token), split[1:], token, distance)
                        entity = row.text[result[0]:result[1]+1]
                        holder.append(x)

                    else:
                        result = singlematch(token.idx, token.idx + len(token), x, token, distance)
                        entity = row.text[result[0]:result[1]+1]
                        holder.append(x)
                            
    if len(set([x.lower() for x in true if x != None])) == 1 and len(set(holder)) == 1:
        return set(holder + ['none'])
    if None in true and len(set(holder)) == 1:
        return set(holder + ['none'])
    return set(holder)

def fix_entries_ships(row):
    true_data_matches = {}
    if row.data_matches != 0:
        for x in row.data_matches:
            ships = entity_finder(row, 80, [x['ship_out'], x['ship_return']])
            rank =  entity_finder(row, 80, [x['rank']])
            location = entity_finder(row, 80, [x['place_of_origin']])
            true_data_matches[str(x['index'])] = {'ships': ships, 'location':location, 'rank':rank}
    return true_data_matches

In [77]:
ship_matches = [fix_entries_ships(x) for x in notebook.tqdm(final_df.itertuples(), total=len(final_df))]
    

In [10]:
voc = pd.read_csv('../vocop-clustered-new.csv', sep='	')

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (17,27,35,36,38) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


777

In [13]:
voc.fullNameNormalized.value_counts()

Jan Jans                   2107
Jan Piters                 1606
Pieter Jans                1383
Hendrik Jans                977
Jan Hendriks                868
                           ... 
Pieter Abjate Barnabas        1
Augustinus Abraham Rosa       1
Adrianus de Klerg             1
Johan Kristoffel Reppi        1
Hermanus Murk                 1
Name: fullNameNormalized, Length: 493653, dtype: int64

In [7]:
final_df

,uuid,rubriek,notaris,inventarisNr,akteNr,akteType,datering,taal,beschrijving,namen,urls,text,name,data_matches,data_entry,vocop_match
0,17599c0c-3305-165c-aae3-eddbb497e4b4,358,JAN VERLEIJ,11885,107542,Testament,1741-03-16,nederlands,None,"[{'voornaam': 'Trijntje', 'tussenvoegsel': Non...","['KLAB05372000012.JPG', 'KLAB05372000013.JPG',...",N: 4 1e de Testateuren hebben verklaerd te Sam...,Trijntje Lansink,0,0,0
1,17599c0c-3305-165c-aae3-eddbb497e4b4,358,JAN VERLEIJ,11885,107542,Testament,1741-03-16,nederlands,None,"[{'voornaam': 'Trijntje', 'tussenvoegsel': Non...","['KLAB05372000012.JPG', 'KLAB05372000013.JPG',...",N: 4 1e de Testateuren hebben verklaerd te Sam...,Martinus Ciprianus,0,0,0
2,17599c0c-3305-165c-aae3-eddbb497e4b4,358,JAN VERLEIJ,11885,107542,Testament,1741-03-16,nederlands,None,"[{'voornaam': 'Trijntje', 'tussenvoegsel': Non...","['KLAB05372000012.JPG', 'KLAB05372000013.JPG',...",N: 4 1e de Testateuren hebben verklaerd te Sam...,Simon van Zuijderwijk,0,0,0
3,17599c0c-3305-165c-aae3-eddbb497e4b4,358,JAN VERLEIJ,11885,107542,Testament,1741-03-16,nederlands,None,"[{'voornaam': 'Trijntje', 'tussenvoegsel': Non...","['KLAB05372000012.JPG', 'KLAB05372000013.JPG',...",N: 4 1e de Testateuren hebben verklaerd te Sam...,Margareta van Zuijdewijk,0,0,0
4,1b037028-bea5-0fa8-e6c5-68155f5f2b21,358,JAN VERLEIJ,11885,108046,Attestatie,1741-07-26,nederlands,\nordentelijke vrijage\n,"[{'voornaam': 'Gerard', 'tussenvoegsel': None,...","['KLAB05372000193.JPG', 'KLAB05372000194.JPG',...",Verklaing gepasseert den 26 Julij 1741No: 60- ...,Gerard Plaatman,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3129,ffb782b0-4689-9a39-e3df-e4b36397882d,358,JAN VERLEIJ,11919,15522,Wisselprotest,1753-02-08,nederlands,None,"[{'voornaam': 'Coenraad', 'tussenvoegsel': Non...","['KLAB05698000178.JPG', 'KLAB05698000179.JPG',...",150:-No:88 Wisel Protest gedaan den 8e: febr: ...,Coenraad Smit,"[{'index': 83363, 'name_original': 'Coenraad S...","{'83363': {'ships': ['nan'], 'rank': [], 'loca...",0
3130,ffb782b0-4689-9a39-e3df-e4b36397882d,358,JAN VERLEIJ,11919,15522,Wisselprotest,1753-02-08,nederlands,None,"[{'voornaam': 'Coenraad', 'tussenvoegsel': Non...","['KLAB05698000178.JPG', 'KLAB05698000179.JPG',...",150:-No:88 Wisel Protest gedaan den 8e: febr: ...,Johann Wilhelm Schmittener,0,0,0
3131,ffb782b0-4689-9a39-e3df-e4b36397882d,358,JAN VERLEIJ,11919,15522,Wisselprotest,1753-02-08,nederlands,None,"[{'voornaam': 'Coenraad', 'tussenvoegsel': Non...","['KLAB05698000178.JPG', 'KLAB05698000179.JPG',...",150:-No:88 Wisel Protest gedaan den 8e: febr: ...,Jochem Matthijs Smit,0,0,0
3132,ffb782b0-4689-9a39-e3df-e4b36397882d,358,JAN VERLEIJ,11919,15522,Wisselprotest,1753-02-08,nederlands,None,"[{'voornaam': 'Coenraad', 'tussenvoegsel': Non...","['KLAB05698000178.JPG', 'KLAB05698000179.JPG',...",150:-No:88 Wisel Protest gedaan den 8e: febr: ...,Jan Salm,"[{'index': 59938, 'name_original': 'Jan Sam', ...","{'59938': {'ships': ['nan'], 'rank': [], 'loca...",0


In [96]:
for x in range(final_df.shape[0]):
    final_df.iloc[x].data_entry = ship_matches[x]

In [6]:
final_df

,uuid,rubriek,notaris,inventarisNr,akteNr,akteType,datering,taal,beschrijving,namen,urls,text,name,data_matches,data_entry,vocop_match
0,17599c0c-3305-165c-aae3-eddbb497e4b4,358,JAN VERLEIJ,11885,107542,Testament,1741-03-16,nederlands,None,"[{'voornaam': 'Trijntje', 'tussenvoegsel': Non...","['KLAB05372000012.JPG', 'KLAB05372000013.JPG',...",N: 4 1e de Testateuren hebben verklaerd te Sam...,Trijntje Lansink,0,0,0
1,17599c0c-3305-165c-aae3-eddbb497e4b4,358,JAN VERLEIJ,11885,107542,Testament,1741-03-16,nederlands,None,"[{'voornaam': 'Trijntje', 'tussenvoegsel': Non...","['KLAB05372000012.JPG', 'KLAB05372000013.JPG',...",N: 4 1e de Testateuren hebben verklaerd te Sam...,Martinus Ciprianus,0,0,0
2,17599c0c-3305-165c-aae3-eddbb497e4b4,358,JAN VERLEIJ,11885,107542,Testament,1741-03-16,nederlands,None,"[{'voornaam': 'Trijntje', 'tussenvoegsel': Non...","['KLAB05372000012.JPG', 'KLAB05372000013.JPG',...",N: 4 1e de Testateuren hebben verklaerd te Sam...,Simon van Zuijderwijk,0,0,0
3,17599c0c-3305-165c-aae3-eddbb497e4b4,358,JAN VERLEIJ,11885,107542,Testament,1741-03-16,nederlands,None,"[{'voornaam': 'Trijntje', 'tussenvoegsel': Non...","['KLAB05372000012.JPG', 'KLAB05372000013.JPG',...",N: 4 1e de Testateuren hebben verklaerd te Sam...,Margareta van Zuijdewijk,0,0,0
4,1b037028-bea5-0fa8-e6c5-68155f5f2b21,358,JAN VERLEIJ,11885,108046,Attestatie,1741-07-26,nederlands,\nordentelijke vrijage\n,"[{'voornaam': 'Gerard', 'tussenvoegsel': None,...","['KLAB05372000193.JPG', 'KLAB05372000194.JPG',...",Verklaing gepasseert den 26 Julij 1741No: 60- ...,Gerard Plaatman,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3129,ffb782b0-4689-9a39-e3df-e4b36397882d,358,JAN VERLEIJ,11919,15522,Wisselprotest,1753-02-08,nederlands,None,"[{'voornaam': 'Coenraad', 'tussenvoegsel': Non...","['KLAB05698000178.JPG', 'KLAB05698000179.JPG',...",150:-No:88 Wisel Protest gedaan den 8e: febr: ...,Coenraad Smit,"[{'index': 83363, 'name_original': 'Coenraad S...","{'83363': {'ships': ['nan'], 'rank': [], 'loca...",0
3130,ffb782b0-4689-9a39-e3df-e4b36397882d,358,JAN VERLEIJ,11919,15522,Wisselprotest,1753-02-08,nederlands,None,"[{'voornaam': 'Coenraad', 'tussenvoegsel': Non...","['KLAB05698000178.JPG', 'KLAB05698000179.JPG',...",150:-No:88 Wisel Protest gedaan den 8e: febr: ...,Johann Wilhelm Schmittener,0,0,0
3131,ffb782b0-4689-9a39-e3df-e4b36397882d,358,JAN VERLEIJ,11919,15522,Wisselprotest,1753-02-08,nederlands,None,"[{'voornaam': 'Coenraad', 'tussenvoegsel': Non...","['KLAB05698000178.JPG', 'KLAB05698000179.JPG',...",150:-No:88 Wisel Protest gedaan den 8e: febr: ...,Jochem Matthijs Smit,0,0,0
3132,ffb782b0-4689-9a39-e3df-e4b36397882d,358,JAN VERLEIJ,11919,15522,Wisselprotest,1753-02-08,nederlands,None,"[{'voornaam': 'Coenraad', 'tussenvoegsel': Non...","['KLAB05698000178.JPG', 'KLAB05698000179.JPG',...",150:-No:88 Wisel Protest gedaan den 8e: febr: ...,Jan Salm,"[{'index': 59938, 'name_original': 'Jan Sam', ...","{'59938': {'ships': ['nan'], 'rank': [], 'loca...",0


In [15]:
ranking_df = pd.DataFrame(columns=['name_ratio', 'name_count', 'day_dif', 'location', 'rank', 'numships', 'match', 'notary_id', 'voc_id'])
name_dict = voc.fullNameNormalized.value_counts()
for x in notebook.tqdm(final_df[final_df.data_matches != 0].itertuples(), total=final_df[final_df.data_matches != 0].shape[0]):
    notary_date = datetime.strptime(x.datering, '%Y-%m-%d')
    notid = x.Index
    for match in x.data_matches:
        index = str(x.Index) + '_' + str(match['index'])
        vocid=match['index']
        name_ratio=max(fuzz.ratio(x.name, match['name_original']), fuzz.ratio(x.name, match['name_normalized']))
        name_count = name_dict[match['name_normalized']]
        
        if x.vocop_match != 0 and match['index'] == x.vocop_match[1]:
            matched = 1
        else:
            matched = 0
        
        try:
            out_date = datetime.strptime(match['date_out'], '%Y-%m-%d')
        except:
            out_date = datetime(year=1, month=1, day =1 )
        try:
            return_date = datetime.strptime(match['date_return'], '%Y-%m-%d')
        except:
            return_date = datetime(year=1, month=1, day =1 )
        if (notary_date - out_date).days in range(0, -91, -1):
            day_dif = -(notary_date - out_date).days
        elif (notary_date - return_date).days in range(0, 91):
            day_dif = (notary_date - return_date).days
        else:
            continue
        
        if str(match['index']) in x.data_entry:
            numships = len(x.data_entry[str(match['index'])]['ships'])
            if x.data_entry[str(match['index'])]['rank'] != []:
                rank = 1
            else:
                rank = 0
            if x.data_entry[str(match['index'])]['location'] != []:
                location = 1
            else:
                location = 0
        else:
            numships = 0
            rank = 0
            location = 0
        tempdf = pd.DataFrame(columns=['name_ratio', 'name_count', 'day_dif', 'location', 'rank', 'numships', 'match', 'notary_id', 'voc_id'], data={
            'name_ratio':name_ratio,
            'name_count':name_count,
            'day_dif':day_dif, 
            'location':location,
            'rank':rank,
            'numships':numships,
            'match':matched,
            'notary_id':notid,
            'voc_id':vocid
        }, index=[index])
        ranking_df = ranking_df.append(tempdf)

In [16]:
ranking_df

,name_ratio,name_count,day_dif,location,rank,numships,match,notary_id,voc_id
14_62148,92,2,18,0,0,0,0,14,62148
16_264891,90,1,60,1,0,0,0,16,264891
16_295399,90,6,26,0,0,0,0,16,295399
48_186881,90,20,16,1,0,0,0,48,186881
52_379356,90,51,8,0,0,1,0,52,379356
...,...,...,...,...,...,...,...,...,...
3112_428673,100,2107,3,0,0,0,0,3112,428673
3112_435880,95,2107,85,0,0,0,0,3112,435880
3112_457632,100,2107,3,0,0,0,0,3112,457632
3112_471148,100,2107,64,1,0,0,0,3112,471148


In [17]:
ranking_df.to_csv('../preranking.csv', index=False)
#final_df.to_csv('../final_df.csv')
#ranking_df[ranking_df.match == 1]